In [ ]:
import pandas as pd
from openbb_terminal.api import openbb as obb
from company import Company

In [ ]:
peers = [('MSFT', 1), ('SPB', 2), ('SPCE', 3), ('ETSY', 4)]

# function used to convert strings representing percentage to a float, eg '10%' becomes 0.1
def p2f(string_perc):
    return float(string_perc.strip('%'))/100

for company in peers:
    
    stock = company[0]
    slot = company[1]

############## *** Making calls to OpenBB to get collections of data: *** ###################
# Data sequences in this section will be massaged and used later to set company object variables
  
    df_metrics = obb.stocks.fa.fmp_metrics(stock) # KEYS CALLED LATER: 'Book value per share', 'Ptb ratio', 'Pe ratio', 
                                                  # 'Pfcf ratio', 'Dividend yield', 'Roe', 'Return on tangible assets', 
                                                  # 'Current ratio', 'Debt to equity', 'Price to sales ratio'
                                                                           
    # fmp's api will sometimes throw error and return an empty list instead of requested dataframe      
    if not isinstance(df_metrics, pd.DataFrame):
        raise TypeError('Dag nabbit!' + ' ' + stock + ' is not available via FMP api. Failed to build df_metrics')
        
    # Where possible, converting strings to floats in df_metrics    
    float_error_set = set() 
    for column in df_metrics.columns:
        
        for row in df_metrics.index:
            
            try:
                df_metrics[column][row] = float(df_metrics[column][row])
                
            except ValueError: # catching the row names for those that cannot be converted to float
                float_error_set.add(row)
                
    df_metrics_discarded = pd.DataFrame() # holds data removed from df_metrics in case needed later
    for row in float_error_set: # Adding appropriate rows to the new dataframe
        df_metrics_discarded[row] = df_metrics.loc[row]

    df_metrics_discarded = df_metrics_discarded.T # transposing so it matches format of df_metrics
    
    # Removing any row from df_metrics where values cannot be converted to float
    df_metrics = df_metrics.drop(index= float_error_set)
    
    # Creating new column that will hold the average of each row
    df_metrics['5yr Avg'] = df_metrics.mean(axis=1)

    # Getting Stock Twits sentiment data as a tuple:
    tuple_twits = obb.stocks.ba.bullbear(stock)
    
    # Analyst recommendation totals by type over last 3 months
    df_rot = obb.stocks.dd.rot(stock).T[0]
#    df_rot = df_rot.T
#    df_rot = df_rot[0]

################################################################################################    
# Creating variables for readability then grouping values into DataFrames & Sequences
################################################################################################         
    
############## *** BASIC DETAILS DATAFRAME *** ##############

    name = obb.stocks.fa.profile(stock).loc['companyName'][0]
    ticker = stock
    sector = obb.stocks.fa.profile(stock).loc['sector'][0]
    industry = obb.stocks.fa.profile(stock).loc['industry'][0]
    cap = obb.stocks.fa.quote(stock).loc['Market cap'][0]
    price = obb.stocks.fa.quote(stock).loc['Price'][0]
    
    # DataFrame will be added to company object
    df_basic = pd.DataFrame({'name': name, 'ticker': ticker, 'sector': sector,
                             'industry': industry, 'cap': cap, 'price': price}, 
                              index=['Basic Details']).T
    
############## *** VALUE METRICS DATAFRAME *** ##############

    try: # If company has no long term debt 0 will be assigned to associated variable
        tca = obb.stocks.fa.yf_financials(stock, "balance-sheet").loc['Total current assets'][0]
        tld = obb.stocks.fa.yf_financials(stock, "balance-sheet").loc['Long-term debt'][0]
        
    except KeyError:
#        tca = obb.stocks.fa.yf_financials(stock, "balance-sheet").loc['Total current assets'][0]
        tld = 0     

    tca_m_tld = tca - tld    

    ptb_mrq = float(obb.stocks.fa.data(stock).loc['P/B'][0])
    ptb_ttm = df_metrics.loc['Ptb ratio'][0]
    ptb_5yr_avg = df_metrics.loc['Ptb ratio']['5yr Avg']

    bvps_mrq = float(obb.stocks.fa.data(stock).loc['Book/sh'][0])
    bvps_ttm = df_metrics.loc['Book value per share'][0]
    bvps_5yr_avg = df_metrics.loc['Book value per share']['5yr Avg']
    
    try: # Because some companies do not have earnings
        pe_mrq = float(obb.stocks.fa.data(stock).loc['P/E'][0])
        pe_ttm = df_metrics.loc['Pe ratio'][0]
        pe_5yr_avg = df_metrics.loc['Pe ratio']['5yr Avg']
        
    except KeyError:
        pe_mrq = 'n/a'
        pe_ttm = 'n/a'
        pe_5yr_avg = 'n/a'
        
    try: 
        pfcf_mrq = float(obb.stocks.fa.data(stock).loc['P/FCF'][0])
        pfcf_ttm = df_metrics.loc['Pfcf ratio'][0]
        pfcf_5yr_avg = df_metrics.loc['Pfcf ratio']['5yr Avg']
        
    except KeyError:
        pfcf_mrq = 'n/a'
        pfcf_ttm = 'n/a'
        pfcf_5yr_avg = 'n/a'
    
    try: # Because some companies do not have sales
        pts_mrq = float(obb.stocks.fa.data(stock).loc['P/S'][0])
        pts_ttm = df_metrics.loc['Price to sales ratio'][0]
        pts_5yr_avg = df_metrics.loc['Price to sales ratio']['5yr Avg']
        
    except KeyError:
        pts_mrq = 'n/a'
        pts_ttm = 'n/a'
        pts_5r_avg = 'n/a'

    # DataFrame will be added to company object   
    df_value = pd.DataFrame({'tca': tca, 'tld': tld, 'tca_m_tld': tca_m_tld, 'ptb_mrq': ptb_mrq, 
                             'ptb_ttm': ptb_ttm, 'ptb_5yr_avg': ptb_5yr_avg, 'bvps_mrq': bvps_mrq, 
                             'bvps_ttm': bvps_ttm, 'bvps_5yr_avg': bvps_5yr_avg, 'pe_mrq': pe_mrq, 
                             'pe_ttm': pe_ttm, 'pe_5yr_avg': pe_5yr_avg, 'pfcf_mrq': pfcf_mrq, 
                             'pfcf_ttm': pfcf_ttm, 'pfcf_5yr_avg': pfcf_5yr_avg, 'pts_mrq': pts_mrq,
                             'pts_ttm': pts_ttm, 'pts_5yr_avg': pts_5yr_avg}, 
                              index=['Value Metrics']).T
        
############## *** MANAGEMENT METRICS DATAFRAME *** ##############

    roe_mrq = p2f(obb.stocks.fa.data(stock).loc['ROE'][0])
    roe_ttm = df_metrics.loc['Roe'][0]
    roe_5yr_avg = df_metrics.loc['Roe']['5yr Avg']
    
    roa_mrq = p2f(obb.stocks.fa.data(stock).loc['ROA'][0])
    roa_ttm = df_metrics.loc['Return on tangible assets'][0]
    roa_5yr_avg = df_metrics.loc['Return on tangible assets']['5yr Avg']
    
    gpr = float(obb.stocks.fa.fmp_income(stock).loc['Gross profit ratio'][0])
    
    try: # Some companies are do not have profit margin
        pm = p2f(obb.stocks.fa.data(stock).loc['Profit Margin'][0])
        
    except KeyError:
        pm = '0%'

    cr_mrq = float(obb.stocks.fa.data(stock).loc['Current Ratio'][0])
    cr_ttm = df_metrics.loc['Current ratio'][0]
    cr_5yr_avg = df_metrics.loc['Current ratio']['5yr Avg']
    
    dte_mrq = float(obb.stocks.fa.data(stock).loc['LT Debt/Eq'][0])
    dte_ttm = df_metrics.loc['Debt to equity'][0]
    dte_5yr_avg = df_metrics.loc['Debt to equity']['5yr Avg']
    
    # DataFrame will be added to company object    
    df_mgmt = pd.DataFrame({'roe_mrq': roe_mrq, 'roe_ttm': roe_ttm, 'roe_5yr_avg': roe_5yr_avg,
                            'roa_mrq': roa_mrq, 'roa_ttm': roa_ttm, 'roa_5yr_avg': roa_5yr_avg,
                            'gpr': gpr, 'pm': pm, 'cr_mrq': cr_mrq, 'cr_ttm': cr_ttm, 
                            'cr_5yr_avg': cr_5yr_avg, 'dte_mrq': dte_mrq, 'dte_ttm': dte_ttm,
                            'dte_5yr_avg': dte_5yr_avg}, index=['Management Metrics']).T
        
############## *** INSIDER & INSTITUION DATAFRAME *** ##############

    io = p2f(obb.stocks.fa.data(stock).loc['Insider Own'][0])
    it = p2f(obb.stocks.fa.data(stock).loc['Insider Trans'][0])
    
    try:
        inst_o = p2f(obb.stocks.fa.data(stock).loc['Inst Own'][0])
        
    except KeyError:
        inst_o = 'n/a'
     
    try:
        inst_t = p2f(obb.stocks.fa.data(stock).loc['Inst Trans'][0])
        
    except KeyError:
        inst_t = 'n/a'

    # DataFrame will be added to company object
    df_ins = pd.DataFrame({'io': io, 'it': it, 'inst_o': inst_o, 
                           'inst_t': inst_t}, index=['Insider & Insitution Data']).T
    
############## *** DIVIDEND DATAFRAMES *** ##############
# Note that there are 2 Dividend DataFrames: df_div & df_div_his

    try: # If company provides no dividend 0 will be assigned
        div = obb.stocks.fa.data(stock).loc['Dividend'][0]
    
    except KeyError:
        div = 0
   
    try: # If company provides no dividend 0 will be assigned
        div_y = df_metrics.loc['Dividend yield'][0]
        
    except KeyError:
        div_y = 0
    
    # DataFrame will be added to company object via div_dfs list
    df_div = pd.DataFrame({'div': div, 'div_y': div_y}, index=['Dividend Metrics']).T
    
    # DataFrame (or string if no dividend) will be added to company object via div_dfs list
    df_div_his = obb.stocks.fa.divs(stock)    
    if df_div_his.empty:
        df_div_his = 'n/a'
    
    # List will be added to company object
    div_dfs = [df_div, df_div_his]
    
############## *** PUBLIC SENTIMENT & SHORT INTEREST DATAFRAME *** ##############
    
    if tuple_twits[1] != 0 and tuple_twits[2] != 0: # Normal case
        twits_perc = tuple_twits[2] / tuple_twits[1]
        
    elif tuple_twits[1] == 0: # Case for if there are no ratings
        twits_perc = 'Not Rated'
        
    elif tuple_twits[2] == 0: # Case for if all ratings are bearish
        twits_perc = 0
        
    # Getting sentiment from last 10 days of news articles
    df_news_sent = obb.stocks.ba.headlines(stock)

    sent_list = []
    for date in df_news_sent.index:
        sent_list.append(float(df_news_sent.loc[date][0]))
        
    # Calculating the average sentiment over last 10 days
    news_sent = sum(sent_list)/len(sent_list)
        
    shrt_int = p2f(obb.stocks.fa.data(stock).loc['Short Float'][0])
    
    # DataFrame will be added to company object
    df_pub_sent = pd.DataFrame({'twits_perc': twits_perc, 'shrt_int': shrt_int, 
                                'news_sent': news_sent}, index=['Public Sentiment Metrics']).T
        
############## *** COMPANY, SECTOR, INDUSTRY NEWS & SENTIMENT *** ##############
# Note there are 3 DataFrames here: df_company_news, df_sector_news, df_industry_news

    # using df_profile to get companyName used for search string
    df_com_news = obb.common.news(name, sort='published').head(20)       
    
    # using df_profile to get sector used for search string
    df_sec_news = obb.common.news(sector + 'Sector News Stock Market', sort='published').head(50)
    
    # using df_profile to get industry used for search string
    df_ind_news = obb.common.news(industry + 'Industry News Stock Market', sort='published').head(50) 
    
    # List will be added to company object
    news_dfs = [df_com_news, df_sec_news, df_ind_news]
    
############## *** ANALYST RATINGS *** ##############
# Note there are 2 DataFrames and one variable here: df_rating_30d, df_rot_3mo, wb_score
  
    # Pairing df_rating down to last 30 days and converting returned Series object back into DataFrame
    # DataFrame will be added to company object
    df_rating_30d = obb.stocks.dd.rating(stock).T.loc['Rating'].head(30).to_frame()
    
    # DataFrame will be added to company object
    df_rot_3mo = pd.DataFrame({'Strong Buy': df_rot['strongBuy'], 'Buy': df_rot['buy'], 'Hold': df_rot['hold'], 
                           'Sell': df_rot['sell'], 'Strong Sell': df_rot['strongSell']}, index=['Last 3mo']).T

    # Warren Buffet Investing Score
    # Variable will be added to company object
    wb_score = obb.stocks.fa.score(stock)
    if wb_score is None:
        wb_score = 'n/a'       

    analyst_data = [df_rating_30d, df_rot_3mo, wb_score]
############## *** ESG RATINGS *** ##############

    df_esg = obb.stocks.fa.sust(stock)
    
##################################################################################    
############## *** Creating Company objects: *** #################################  
##################################################################################   
    if slot == 1:

        c1 = Company(df_basic, df_value, df_mgmt, df_ins,
                           div_dfs, df_pub_sent, news_dfs, 
                           analyst_data, df_esg)
        
    elif slot == 2:

        c2 = Company(df_basic, df_value, df_mgmt, df_ins,
                           div_dfs, df_pub_sent, news_dfs, 
                           analyst_data, df_esg)
        
    elif slot == 3:

        c3 = Company(df_basic, df_value, df_mgmt, df_ins,
                           div_dfs, df_pub_sent, news_dfs, 
                           analyst_data, df_esg)
        
    elif slot == 4:

        c4 = Company(df_basic, df_value, df_mgmt, df_ins,
                           div_dfs, df_pub_sent, news_dfs, 
                           analyst_data, df_esg)

In [ ]:
# use this Company method to combine selected dataframes and export them into a seperate excel file for each company:
for company in [c1, c2, c3, c4]:
    company.data_to_excel()

In [ ]:
# use this Company method to display selected company dataframes
for company in [c1, c2, c3, c4]:
    company.display_dfs()